**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_squared_error,
                             mean_absolute_error)
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from class_utils import corr_heatmap, error_histogram
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/8s0ivlo9yshhxkn/winequality.zip?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

### Lineárna regresia a kvalita vína

Ďalej si vyskúšame lineárnu regresiu na dátovej množine na predikciu kvality bieleho vína.

Načítame dátovú množinu z CSV súboru:



In [ ]:
df = pd.read_csv("data/winequality-white.csv")
df.head()

#### Obsahuje dátová množina lineárne závislosti?

Aby sme zistili, či dátová množina obsahuje nejaké lineárne závislosti, ktoré sa budú dať lineárnou regresiou modelovať, zobrazíme si korelačnú maticu. Silne korelované premenné majú jasnú lineárnu závislosť. Silná záporná korelácia tiež znamená, že premenné sú lineárne závislé: ibaže sú jedna druhej nepriamo úmerné. Niektoré polia korelačnej matice sú biele: znamená to, že korelácia nebola štatisticky významná a dané čísla nemajú veľkú výpovednú hodnotu.



In [ ]:
plt.figure(figsize=(10, 8))
corr_heatmap(df)
plt.savefig("output/wine_corr_matrix.pdf", bbox_inches="tight", pad_inches=0)

Ako vidno z grafu, existuje pomerne silná korelácia medzi premennými `density` a `residual sugar`. Tieto premenné majú slabšiu koreláciu aj s viacerými ďalšími. Môžeme sa teda pokúsiť predikovať premennú `density` z ostatných pomocou lineárnej regresie.

Ak by sme chceli predikovať kvalitu vína, lineárna regresia pravdepodobne nebude veľmi dobrý spôsob: silnejšia korelácia je len s obsahom alkoholu.

### Predspracovanie

Rozdeľme si teraz dátovú množinu na tréningovú a testovaciu časť so stratifkáciou podľa atribútu `density` a aplikujme štandardné predspracovanie.



In [ ]:
#@title -- Dataset Splittling: df_train, df_test -- { display-mode: "form" }
kbins = KBinsDiscretizer(6, encode='ordinal')
y_stratify = kbins.fit_transform(df[['density']])
df_train, df_test = train_test_split(df, stratify=y_stratify,
                                 test_size=0.3, random_state=4)

In [ ]:
df.head()

---
### Úloha 1: Typy stĺpcov

**Nižšie uveďte, ktoré kategorické a numerické stĺpce sa majú použiť.** 

---


In [ ]:
categorical_inputs = [           ]  # ----

numeric_inputs = [               ]  # ----

output = ['density']

In [ ]:
#@title -- Our Standard Preprocessing: X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

X_train = input_preproc.fit_transform(df_train)
Y_train = df_train[output].values

X_test = input_preproc.transform(df_test)
Y_test = df_test[output].values

#### Ladenie modelu

Pomocou tréningových dát naladíme lineárny model:



In [ ]:
model = LinearRegression()
model = model.fit(X_train, Y_train)

#### Testovanie modelu

Model otestujeme na testovacích dátach:



In [ ]:
#@title -- Testing -- { display-mode: "form" }
y_test = model.predict(X_test)

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_test, y_test)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_test, y_test)
print("MAE = {}".format(mae))

plt.figure(figsize=(8, 6))
error_histogram(Y_test, y_test, Y_fit_scaling=Y_train)

#### Len s použitím vstupu `residual sugar`



In [ ]:
categorical_inputs = []
numeric_inputs = ["residual sugar"]
output = ['density']

In [ ]:
#@title -- Our Standard Preprocessing: X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

X_train = input_preproc.fit_transform(df_train)
Y_train = df_train[output].values

X_test = input_preproc.transform(df_test)
Y_test = df_test[output].values

In [ ]:
model = LinearRegression()
model = model.fit(X_train, Y_train)

In [ ]:
#@title -- Testing -- { display-mode: "form" }
y_test = model.predict(X_test)

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_test, y_test)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_test, y_test)
print("MAE = {}".format(mae))

plt.figure(figsize=(8, 6))
error_histogram(Y_test, y_test, Y_fit_scaling=Y_train)

Ako vidno, výsledky sú v tomto prípade o dosť horšie. Zdá sa, že ostatné stĺpce obsahujú informácie o hustote, ktoré sa nedajú vyčítať len zo stĺpca `residual sugar`.

